<a href="https://colab.research.google.com/github/JIWON0520/TIL/blob/main/Paper%20Review/Alexnet_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

In [ ]:
!mkdir /content/drive/MyDrive/Alexnet

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
#image 256X256사이즈로 변경후 227X227로 crop
transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.RandomCrop(227),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset=datasets.CIFAR10('/content/drive/MyDrive/Alexnet',download=True,train=True,transform=transform)
trainset=DataLoader(trainset,batch_size=256,shuffle=True)
testset=datasets.CIFAR10('/content/drive/MyDrive/Alexnet',download=True,train=False,transform=transform)
testset=DataLoader(testset,batch_size=128,shuffle=True)

In [4]:
class AlexNet(nn.Module):
  def __init__(self,num_classes=10):
    super(AlexNet,self).__init__()

    self.convLayer=nn.Sequential(
        #Conv 1: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,padding=0,stride=4),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2),

        #Conv 2: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,padding=2,stride=1),
        nn.ReLU(),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2),

        #Conv 3: Conv,ReLU
        nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,padding=1,stride=1),
        nn.ReLU(),

        #Conv 4: ConV,ReLU
        nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,padding=1,stride=1),
        nn.ReLU(),

        #Conv 5: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),
    )

    self.fcLayer=nn.Sequential(
        #fc 1: fc,ReLU,dropout
        nn.Linear(in_features=(256*6*6),out_features=4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),

        #fc 2:fc,ReLU,dropout
        nn.Linear(in_features=4096,out_features=4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),

        #fc 3:fc
        nn.Linear(in_features=4096,out_features=num_classes),
    )

    for layer in self.convLayer:
      if isinstance(layer,nn.Conv2d):
        nn.init.normal_(layer.weight,mean=0,std=0.01)
        nn.init.constant_(layer.bias,0)
      elif isinstance(layer,nn.Linear):
        nn.init.normal_(layer.weight,mean=0,std=0.01)
        nn.init.constant_(layer.bias,1)

    nn.init.constant_(self.convLayer[4].bias,1)
    nn.init.constant_(self.convLayer[10].bias,1)
    nn.init.constant_(self.convLayer[12].bias,1)

  def forward(self,x):
    x=self.convLayer(x)
    x=x.view(-1,256*6*6)
    x=self.fcLayer(x)
    return x

In [6]:
alexnet=AlexNet(num_classes=10).to(device)

#Optimizer 생성
#learning rate=0.01, momentum=0.9, weigth decay=0.0005
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9, weight_decay=0.0005)
criterion=nn.CrossEntropyLoss()

In [ ]:
alexnet.train()
lr_scheduler=optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma=0.1)
epochs=90
iteration=len(trainset)
for epoch in range(epochs):
  epoch_loss=0
  for data,classes in trainset:
    inputs,labels=data.to(device),classes.to(device)
    
    optimizer.zero_grad()
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    epoch_loss+=loss.item()
  lr_scheduler.step()

  print('epoch{0} loss:{1:.3f}'.format(epoch,epoch_loss/iteration))


In [ ]:
alexnet.eval()

with torch.no_grad():
  total=0.0
  accuracy=0.0
  for data,classes in testset:
    inputs,labels=data.to(device),classes.to(device)

    outputs=alexnet(inputs)

    _,pred=torch.max(outputs.data,1)
    total+=outputs.size(0)
    accuracy+=(pred==labels).sum().item()
print((accuracy/total)*100)